In [1]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from PyPDF2 import PdfReader
from transformers import pipeline

import chromadb
import torch
import ollama

/home/nayra/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

In [3]:
PATH = "pdf/Livro de Arquitetura e organização de computadores.pdf"


reader = PdfReader(PATH)

text_load = ''.join(page.extract_text() for page in reader.pages)

client = chromadb.PersistentClient(path="chromadb")
collection = client.get_or_create_collection(name="vector-langchain")
history_collection = client.get_or_create_collection(name="chat_history")
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

In [4]:
text_split = []
metadata=[]
chunk_size = 432
chunk_overlap = 8

for i in range(0, len(text_load), chunk_size):
    start = i
    end = i + chunk_size
    if start != 0:
        start = start - chunk_overlap
        end =  end + chunk_overlap
    text_split.append(text_load[start:end])
    metadata.append({"name": "book-langchain", "partition": f"{i}"})
embeddings_db = embeddings.encode(text_split)

In [5]:
for i, d in enumerate(text_split):

      collection.add(
              ids=[str(i)],
              embeddings=embeddings_db[[i]],
              documents=[d],
              metadatas=metadata[i]
    )

In [6]:
while True:

    query = []

    prompt = input("Pergunte algo")
    response = embeddings.encode([prompt])
    results = collection.query(
            query_embeddings=response,
            n_results=10,
            where={"name": "book-langchain"}
        )

    data = results['documents'][0]
    distance = results['distances'] 
    
    for k in range(10):
        if distance[0][k] < 0.74:
            query.append(data[k])

    prompt_template = f"O seu único conhecimento é sobre {query}. Agora, responda {prompt}"

    if len(query) == 0:
            prompt_template = "Você é um assistente de chatbot e tem acesso limitado a informações. Nesse caso, diga que não tem conhecimento acerca da pergunta."
    
    output = ollama.generate(
            model="llama2",
            prompt=prompt_template
            )
    print("\n")
    output = output['response']
    print(output)
    print("\n")
    

Pergunte algo o que é memória ram?


[[0.5899770259857178, 0.6298215985298157, 0.630146861076355, 0.6509194374084473, 0.6580712199211121, 0.6639449000358582, 0.6694378852844238, 0.6802554130554199, 0.6965671181678772, 0.6982316374778748]]
['RAM) é uma forma de memória de acesso aleatório usada para a memória cache; veja o Capítulo 5.\n15 A memória flash consiste em uma forma versátil de memória usada tanto por microcontroladores como por memória externa; esse assunto é discutido no Capítulo 6.\nBOKK_STALLINGS.indb   31 01/09/17   09:15Arquitetura e organização de computadores32\n}}Gerenciamento de energia: administra os vários modos de operação do processador e de periféricos em \nbai', ' (EEPROM — do inglês, \nElectrically Erasable PROM)Eletricamente, \nem nível de byte\nMemória flashEletricamente, \nem nível de bloco\nOutra característica distinta da RAM é que ela é volátil. Uma RAM deve receber uma fonte de alimen-\ntação constante. Se a energia for interrompida, os dados são perdidos. Assim, a RAM só pode ser usada co

Pergunte algo o que é arquitetura de computadores?


[[0.498393177986145, 0.5074726939201355, 0.5160396099090576, 0.5549819469451904, 0.5839642286300659, 0.6416518092155457, 0.6577909588813782, 0.6734175682067871, 0.679311990737915, 0.6805151700973511]]
['se em arquitetura e organização de computadores, com particular ênfase no paralelismo explícito na arquitetura de computadores.\nAo longo da década de 1980 e no início da década de 1990, as pesquisas passaram a brotar nesse campo, \ne houve grandes inovações. A maioria delas chegaram ao mercado pelas start-ups de universidades. Contudo, ironicamente, foi a mesma tecnologia que reverteu essa tendência. A diversidade foi gradativamente substituí da', 'DI; FEUSTEL, 1976.)\nArquitetura de computador refere-se aos atributos de um sistema visíveis a um programador ou, em outras \npalavras, aqueles atributos que possuem um impacto direto sobre a execução lógica de um programa. Um termo que é muitas vezes usado de maneira intercambiável com as arquiteturas de computadores é arquitetura de con-j

Pergunte algo o que é docker?


...

Pergunte algo o que é arquitetura?


[[0.7149341702461243, 0.8029831051826477, 0.8667809963226318, 0.8905649185180664, 0.9073631763458252, 0.9193487167358398, 0.9896033406257629, 0.9902974963188171, 1.006562352180481, 1.0125128030776978]]
['DI; FEUSTEL, 1976.)\nArquitetura de computador refere-se aos atributos de um sistema visíveis a um programador ou, em outras \npalavras, aqueles atributos que possuem um impacto direto sobre a execução lógica de um programa. Um termo que é muitas vezes usado de maneira intercambiável com as arquiteturas de computadores é arquitetura de con-junto de instrução (ISA — do inglês, Instruction Set Architecture) . O ISA define os formatos de instruções,']



Arquitetura de computador referência às características visíveis de um sistema de computador ou hardware, que afetam a execução lógica de um programa. Isso inclui atributos como o tipo de processador, memória RAM, ROM, Unidade de Conhecimento (Knowledge Unit), entre outros.

A arquitetura de computador é importante pois define как as inst

Pergunte algo python


[[1.1592448949813843, 1.1616356372833252, 1.2818533182144165, 1.3011623620986938, 1.3044285774230957, 1.3288086652755737, 1.334482192993164, 1.3441367149353027, 1.359619140625, 1.3613083362579346]]
[]



Entendi! Como assistente de chatbot, tenho acesso limitado a informações e não posso fornecer respostas detalhadas ou precisas para perguntas que estão fora do escopo de meus conhecimentos. Por favor, pergunte alguma outra coisa que esteja dentro do meu áudio de conhecimento.




KeyboardInterrupt: Interrupted by user